In [37]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from io import StringIO 
import os
import numbers
import math
import plotly.graph_objects as go
from ipywidgets import VBox, HBox, HTML, ToggleButton, Label, Checkbox, FileUpload, Output, SelectMultiple, Accordion
from IPython.display import clear_output, display
from plotly.graph_objects import Figure, Scatter


# --- Shared State ---
data = []
df = pd.DataFrame()
df_logs = pd.DataFrame()
keywords = ['Hallo ich bin Rodland die Rodel',
            'Section',
            'Voltage',
            'position', 'Pos:',
            'Velocity', 'vel',
            'List',
            'SPINDLE',
            'speedlimit'
           ]

Voltage = Position = Section = Velocity = brake_power = speedlimit = np.nan
BT_members = BT_members_cnt = 0
start_time = None
sled_nr = ''
filename = ''
content = []
logdata = ''
BT_sleds_set = set()  # current set of connected sleds


#options for displaying the table (for debuging purposes)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  

# --- defines ---
def start(change): #startup program
        global sled_name, filename, content
        file_info = upload_widget.value[0]
        decoded = bytes(file_info['content']).decode('utf-8', errors='ignore') #decode the uploaded file into readable data
        content = decoded.splitlines() #seperate each line from data
        filename.value = file_info['name']
        for line in content:
            line.replace('\n' , '')
        clean_lines(content)
        sled_name = grab_name(file_info)
        parsing(content)  
        draw_graph(0)      

def clean_lines(lines): #clean up lines incase they accidentally get seperated 
    i = 0
    for x in lines:
        if lines[i] == '': #if line is empty delete it from index
            lines.pop(i)
            i -= 1
            continue
        if lines[i][0] != '[' or (lines[i][0] == '[' and lines[i][1].isalpha()): #if first character isnt a "[" or if the first character is a "[" but the second character isnt a number add this line to the previous one
            if i != 0: #ignore the first line
                lines[i-1] = lines [i-1] + lines[i]
                lines.pop(i)
                i -= 1
        i += 1
    return lines

def grab_name(file_info): #read name of the sled out of file name
    file_name = file_info["name"] #get the filename out of file_info
    file_name = file_name.split("_") #split the filename into parts
    for part in file_name: #go though all parts in filename and see if it matches the format Rxxx
        match = re.fullmatch(r'R(\d+)', part)
        if match:
            sled_number = int(match.group(1))
            sled_name = 'Sled#' + str(sled_number) + ':' #save sledname to be like this: Sled#014:
            return(sled_name)

def parsing(clean_lines): #getting values out of file
    global Voltage, Position, Section, Velocity, brake_power, BT_members, BT_members_cnt, start_time, df, data, speedlimit, df_logs, BT_sleds_set
    data = []
    Voltage = Position = Section = Velocity = brake_power = speedlimit = np.nan
    BT_members = BT_members_cnt = 0
    start_time = None
    BT_sleds = np.nan
    
    checks = 0
    
    for word in keywords: #for every word we need to check for
        for line in clean_lines: 
            if word in line:
                clean_line = line.replace('[', '').replace(']', '').replace(',', '').replace('\n', '') # remove all "[],\n"
                parts = re.split(r'[ =]', clean_line)
                
                if checks == 0 and parts[0].isdigit(): #set starttime for the first instance it finds "Hallo ich bin Rodland die Rodel"
                    start_time = int(parts[0])
                    continue
                
                try:
                    time = (int(parts[0]) - start_time) / 100 #for every other time value gets found make it relative and convert it to seconds from ms
                except:
                    continue

                indices = [i for i, item in enumerate(parts) if word in item] #get the location of word in line
                try:
                    match word:
                        case "position":
                            Position = next((parts[i + 1] for i in indices if sled_name in parts), np.nan) #only save position if sled name gets mentioned in line
                        case "Pos:":
                            Position = parts[indices[0] + 1] #get position value after word "Pos:" gets mentioned
                        case "Voltage":
                            Voltage = parts[indices[0] + 1].replace('V', '') #save voltage but without V
                        case "Section":
                            Section = parts[indices[0] + 1]
                        case "Velocity":
                            Velocity = parts[indices[0] + 1]
                        case "vel":
                            Velocity = parts[indices[0] + 1]
                        case "List":
                            line_text = line  # use original line with sled info
                        
                            if 'Insert' in parts[indices[0]]:
                                match = re.search(r'Sled#\d+', line_text)
                                if match:
                                    BT_sleds_set.add(match.group())
                                BT_members_cnt += 1
                            elif 'Remove' in parts[indices[0]]:
                                match = re.search(r'Sled#\d+', line_text)
                                if match and match.group() in BT_sleds_set:
                                    BT_sleds_set.remove(match.group())
                                BT_members_cnt -= 1
                        
                            BT_members = BT_members_cnt
                            BT_sleds = ', '.join(sorted(BT_sleds_set))  # store sorted sled list
                        case "SPINDLE": #when SPINDLE gets mentioned save value as brake_power after pos gets mentioned
                            for x, i in enumerate(parts):
                                if 'pos' in i:
                                    brake_power = parts[x + 1]
                                    break
                        case 'speedlimit':
                            if is_float(parts[indices[0] + 1]): #if the value after speedlimit is a float
                                speedlimit = parts[indices[0] + 1] #save it as speedlimit
                except:
                    continue
                data.append({
                    'Time': time, 'Voltage': Voltage, 'Position': Position, 'Section': Section, 'Velocity': Velocity, 'BT_Members': BT_members, 'Brake_Power': brake_power, 'Speedlimit': speedlimit, 'BT_Sleds': BT_sleds,
                }) #add all values to table
                Section = Voltage = Position = Velocity = brake_power = BT_members = speedlimit = np.nan #reset value
        checks += 1
    # Create and clean DataFrame
    df = []
    df = pd.DataFrame(data)

    for col in ['Time', 'Voltage', 'Position', 'Section', 'Velocity', 'BT_Members', 'Brake_Power', 'Speedlimit']:
        df[col] = pd.to_numeric(df[col], errors='coerce') #convert values to int
    
    df = df.sort_values(by='Time').dropna(subset=['Time']) #sort all values by time and remove values that dont have times related to them
    df = df.groupby('Time', as_index=False).first() #group values with the same time

    fill_list() #fill up list when no values given

    #making dataframe for show logdata accordeon
    df_logs = pd.DataFrame(content, columns=['Log Entry']) #save the "Log entry" comumn as its own data frame
    df_logs['Timestamp'] = df_logs['Log Entry'].str.extract(r'\[(\d+)\]').astype(float) #extract the timestamp of the line
    df_logs['Relative Time'] = (df_logs['Timestamp'] - start_time) / 100

    df_logs = df_logs.loc[:, ['Timestamp', 'Relative Time', 'Log Entry']] #rearange all columns in the df

    df_logs['Display Label'] = df_logs['Relative Time'].map(lambda x: f"{x:.2f}") + ' — ' + df_logs['Log Entry'] #display the relative time as a float with 2 decimal numbers seperated by a - and the log entry
    multi_select.options = df_logs['Display Label']

def draw_graph(change):
    my_min = my_min_2 = 0
    my_max = (((df['Time'].max() + 100) // 100) * 100) * 1.20
    my_max_2 = (((df['Position'].max() + 100) // 100) * 100) * 1.15
    
    with graph_1:
        clear_output(wait=True)
        values = multi_select.value

        multi_min = float(values[0].split("—")[0]) if values else 0
        multi_max = float(values[-1].split("—")[0]) if values else 0

        fig = Figure()

        # Highlighted area
        fig.add_shape(type="rect",
                      x0=multi_min, x1=multi_max,
                      y0=0, y1=1,
                      xref='x', yref='paper',
                      fillcolor='yellow', opacity=0.3, line=dict(width=0))

        fig.add_shape(type="line",
                           x0=multi_min, x1=multi_min,
                           y0=0, y1=1,
                           xref='x', yref='paper',
                           line=dict(color='gold', width=1))
        fig.add_shape(type="line",
                           x0=multi_max, x1=multi_max,
                           y0=0, y1=1,
                           xref='x', yref='paper',
                           line=dict(color='gold', width=1))

        # Section == 33 vertical lines
        for t in df[df['Section'] == 33]['Time']:
            fig.add_shape(type="line",
                          x0=t, x1=t,
                          y0=0, y1=1,
                          xref='x', yref='paper',
                          line=dict(color='cyan', dash='dash'))

        # Voltage
        filtered = df[df['Voltage'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Voltage'], name='Voltage (V)', line=dict(color='blue'), hovertemplate='Time: %{x}<br>Voltage: %{y}<extra></extra>', yaxis='y', mode='lines'))

        # Position
        filtered = df[df['Position'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Position'], name='Position (m)', line=dict(color='orange', shape='hv'), hovertemplate='Time: %{x}<br>Position: %{y}<extra></extra>', yaxis='y2', mode='lines'))

        # Section
        filtered = df[df['Section'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Section'], name='Section', line=dict(color='green', shape='hv'), hovertemplate='Time: %{x}<br>Section: %{y}<extra></extra>', yaxis='y3', mode='lines'))

        # Velocity
        filtered = df[df['Velocity'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Velocity'], name='Velocity (km/h)', line=dict(color='red', shape='hv'), hovertemplate='Time: %{x}<br>Velocity: %{y}<extra></extra>', yaxis='y4', mode='lines'))

        # Speedlimit
        filtered = df[df['Speedlimit'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Speedlimit'], name='Speedlimit', line=dict(color='firebrick', shape='hv'), hovertemplate='Time: %{x}<br>Speedlimit: %{y}<extra></extra>', yaxis='y4', mode='lines'))

        # BT Members
        custom = df['BT_Sleds']  # each row has connected sleds
        fig.add_trace(Scatter(x=df['Time'], y=df['BT_Members'], name='BT Members', line=dict(color='hotpink', shape='hv'), customdata=custom, hovertemplate='Time: %{x}<br>Connected: %{customdata}<extra></extra>', yaxis='y5', mode='lines'))

        # Brake Power
        filtered = df[df['Brake_Power'].notna()]
        fig.add_trace(Scatter(x=filtered['Time'], y=filtered['Brake_Power'], name='Brake Power', line=dict(color='purple', shape='hv'), hovertemplate='Time: %{x}<br>Brake_Power: %{y}<extra></extra>', yaxis='y6', mode='lines'))

        # Layout with stacked right-side axes
        fig.update_layout(
            xaxis=dict(title="Time (s)", range=[my_min, my_max]),
            yaxis1=dict(title='Voltage (V)', tickfont=dict(color='blue'), range=[0, df['Voltage'].max()*1.1]),
            yaxis2=dict(title="Position (m)", overlaying='y', side='right', position=0.98, tickfont=dict(color='orange'), range=[0, df['Position'].max()*1.1]),
            yaxis3=dict(title="Section", overlaying='y', side='right', position=0.95, tickfont=dict(color='green'), range=[0, df['Section'].max()*1.1]),
            yaxis4=dict(title="Velocity", overlaying='y', side='right', position=0.92, tickfont=dict(color='red'), range=[0, df['Speedlimit'].max()*1.1]),
            yaxis5=dict(title="BT Members", overlaying='y', side='right', position=0.89, tickfont=dict(color='hotpink'), range=[0, df['BT_Members'].max()*1.1]),
            yaxis6=dict(title="Brake Power", overlaying='y', side='right', position=0.85, tickfont=dict(color='purple'), range=[0, df['Brake_Power'].max()*1.1]),
            template='plotly_white',
            height=500
        )

        fig.show()

    # Graph 2 (Position-based)
    with graph_2:
        clear_output(wait=True)
        multi_min_2 = multi_max_2 = 0
        fig2 = Figure()

        df_pos = df[df['Position'] > 0]

        multi_min = float(values[0].split("—")[0]) if values else 0
        multi_max = float(values[-1].split("—")[0]) if values else 0

        closest_value_min = df[df['Time'] <= multi_min]['Time'].max()
        closest_value_max = df[df['Time'] <= multi_max]['Time'].max()
        
        count = 0
        for i in df['Time']:
            if i == closest_value_min:
                multi_min_2 = df['Position'][count]
            if i == closest_value_min:
                multi_max_2 = df['Position'][count]
                break
            count += 1
            
        fig2.add_shape(type="rect",
                       x0=multi_min_2, x1=multi_max_2,
                       y0=0, y1=1,
                       xref='x', yref='paper',
                       fillcolor='yellow', opacity=0.3, line=dict(width=0))

        fig2.add_shape(type="line",
                           x0=multi_min_2, x1=multi_min_2,
                           y0=0, y1=1,
                           xref='x', yref='paper',
                           line=dict(color='gold', width=1))
        fig2.add_shape(type="line",
                           x0=multi_max_2, x1=multi_max_2,
                           y0=0, y1=1,
                           xref='x', yref='paper',
                           line=dict(color='gold', width=1))

        for pos in df[(df['Section'] == 33) & (df['Position'] > 0)]['Position']:
            fig2.add_shape(type="line",
                           x0=pos, x1=pos,
                           y0=0, y1=1,
                           xref='x', yref='paper',
                           line=dict(color='cyan', dash='dash'))

        filtered = df_pos[df_pos['Voltage'].notna()]
        fig2.add_trace(Scatter(x=filtered['Position'], y=filtered['Voltage'], name='Voltage (V)', line=dict(color='blue'), hovertemplate='Position: %{x}<br>Voltage: %{y}<extra></extra>', yaxis='y', mode='lines'))

        filtered = df_pos[df_pos['Section'].notna()]
        fig2.add_trace(Scatter(x=filtered['Position'], y=filtered['Section'], name='Section', line=dict(color='green', shape='hv'), hovertemplate='Position: %{x}<br>Section: %{y}<extra></extra>', yaxis='y2', mode='lines'))

        filtered = df_pos[df_pos['Velocity'].notna()]
        fig2.add_trace(Scatter(x=filtered['Position'], y=filtered['Velocity'], name='Velocity (km/h)', line=dict(color='red', shape='hv'), hovertemplate='Position: %{x}<br>Velocity: %{y}<extra></extra>', yaxis='y3', mode='lines'))
        
        filtered_limit = df_pos[df_pos['Speedlimit'].notna()]
        fig2.add_trace(Scatter(x=filtered_limit['Position'], y=filtered_limit['Speedlimit'], name='Speedlimit', line=dict(color='firebrick', shape='hv'), hovertemplate='Position: %{x}<br>Speedlimit: %{y}<extra></extra>', yaxis='y3', mode='lines'))

        custom = df['BT_Sleds']
        fig2.add_trace(Scatter(x=df['Position'], y=df['BT_Members'], name='BT Members', line=dict(color='hotpink', shape='hv'), customdata=custom, hovertemplate='Position: %{x}<br>Connected: %{customdata}<extra></extra>', yaxis='y5', mode='lines'))


        filtered = df_pos[df_pos['Brake_Power'].notna()]
        fig2.add_trace(Scatter(x=filtered['Position'], y=filtered['Brake_Power'], name='Brake Power', line=dict(color='purple', shape='hv'), hovertemplate='Position: %{x}<br>Brake Power: %{y}<extra></extra>', yaxis='y5', mode='lines'))

        fig2.update_layout(
            xaxis=dict(title="Distance (m)", range=[my_min_2, my_max_2]),
            yaxis1=dict(title='Voltage (V)', tickfont=dict(color='blue'), range=[0, df_pos['Voltage'].max()*1.1]),
            yaxis2=dict(title="Section", overlaying='y', side='right', position=0.98, tickfont=dict(color='green'), range=[0, df_pos['Section'].max()*1.1]),
            yaxis3=dict(title="Velocity", overlaying='y', side='right', position=0.95, tickfont=dict(color='red'), range=[0, df_pos['Speedlimit'].max()*1.1]),
            yaxis4=dict(title="BT Members", overlaying='y', side='right', position=0.92, tickfont=dict(color='hotpink'), range=[0, df_pos['BT_Members'].max()*1.1]),
            yaxis5=dict(title="Brake Power", overlaying='y', side='right', position=0.88, tickfont=dict(color='purple'), range=[0, df_pos['Brake_Power'].max()*1.1]),
            template='plotly_white',
            height=500
        )

        fig2.show()




        
def data_reset(change): #reset the graphs and data index if called
    global data
    data = []

    with graph_1:
        clear_output(wait=True)
        plt.close()

    with graph_2:
        clear_output(wait=True)
        plt.close()

    draw_graph(0)

def fill_list(): # fill up the empty entries in the list of data
    global df
    
#    print(df)
    for name in df:
#        print(name)
        name_old = 0
        n = 0
        
        if name not in ['Voltage', 'Section', 'BT_Sleds']:
            i=''
            for i in df[name]:
#                print(i)
                if math.isnan(i):
                    i = name_old
#                    print(i)
                    df.loc[n, name] = i
                else:
                    name_old = i
                n += 1
#                print(i)
        elif name == 'Section':
            i = ''
            n = 0
            for i in df['Section']:
                if i == 0:
                    df[n, 'Position']= np.nan
                n += 1
        elif name == 'BT_Sleds':
            i = ''            
            n = 0
            for i in df['BT_Sleds']:
                if df['BT_Members'][n] != 0.0:
                    if df['BT_Sleds'][n] == None or df['BT_Sleds'][n] == '':
                        df.loc[n, 'BT_Sleds'] = df.loc[n - 1, 'BT_Sleds']
                else:
                    df.loc[n, 'BT_Sleds'] = None
                n += 1 
              
    print(df)

def is_float(value): #check if value is not a float
    try:
        float(value)
        return True
    except ValueError:
        return False



def update_popup(change):
    popup_box.clear_output()
    if change['new']:
        with popup_box:
            display(HTML("""
            <div style="
                position: fixed;
                top: 60px;
                right: 30px;
                z-index: 9999;
                background-color: #fefefe;
                border: 2px solid #ddd;
                border-radius: 10px;
                padding: 20px;
                width: 300px;
                box-shadow: 0 0 15px rgba(0,0,0,0.2);
                font-family: sans-serif;
            ">
                <style>
                table {
                  font-family: arial, sans-serif;
                  border-collapse: collapse;
                  width: 100%;
                }
                
                td, th {
                  border: 1px solid #dddddd;
                  text-align: left;
                  padding: 8px;
                }
                
                tr:nth-child(even) {
                  background-color: #dddddd;
                }
                </style>
                </head>
                <body>
                
                <h1>Info</h1>

                <h3> Begriffe: </h3>

                </p> <b> Time </b> – Zeit nach Start der Logdaten aufnahme </p>
                </p> <b> Voltage </b> – Die Spannung der Batterie der Rodel </p>
                </p> <b> Velocity </b> – Geschwindigkeit mit der sich die Rodel bewegt </p>
                </p> <b> Speedlimit </b> – Die maximale Geschwindigkeit die die Rodel haben darf </p>
                </p> <b> Position </b> – Wo sich die Position auf der Strecke befindet </p>
                </p> <b> Section </b> – In welchem Abschnitt sich die Rodel befindet </p>
                </p> <b> BT_Members </b> – Anzahl an Bluetooth Teilnehmern die mit der Rodel verbunden sind </p>
                </p> <b> Brake_Power </b> – Wie sehr die Bremsen geschlossen sind </p>

                <h3> Section info: </h3>
                                
                <table>
                  <tr>
                    <th>Sektion</th>
                    <th style="width:70%">Definition</th>
                  </tr>
                  <tr>
                    <td>0</td>
                    <td>Init</td>
                  </tr>
                  <tr>
                    <td>1</td>
                    <td>Entry</td>
                  </tr>
                  <tr>
                    <td>2</td>
                    <td>Function check</td>
                  </tr>
                  <tr>
                    <td>3</td>
                    <td>Bergfahrt</td>
                  </tr>
                  <tr>
                    <td>4</td>
                    <td>Talfahrt</td>
                  </tr>
                  <tr>
                    <td>5</td>
                    <td>Exit</td>
                  </tr>
                  <tr>
                    <td>6</td>
                    <td>End</td>
                  </tr>
                  <tr>
                    <td>8</td>
                    <td>Motorized mountain ride</td>
                  </tr>
                  <tr>
                    <td>9</td>
                    <td>Motorized creep</td>
                  </tr>
                  <tr>
                    <td>10</td>
                    <td>Track brake</td>
                  </tr>
                  <tr>
                    <td>33</td>
                    <td>Error</td>
                  </tr>
                  <tr>
                    <td>35</td>
                    <td>Sim mode</td>
                  </tr>
                </table>
                
                </body>
            </div>
            """))
    
# --- Widgets ---
#  File Upload + Title
upload_widget = FileUpload(accept='.log', multiple=False, description='Select File')
title = HTML("<h1 style='color: black;'>Logdata analysis</h1>")
filename = Label()
toggle_popup = ToggleButton(value=False, description="ℹ️ Info", button_style='info', layout={'width': '120px'})

header = HBox(
    [VBox([title]), upload_widget, filename, toggle_popup],
    layout={'align_items': 'flex-end', 'padding': '10px 0 20px 0'}
)

#  Graph outputs
graph_1 = Output(layout={'width': '100%'})
graph_2 = Output(layout={'width': '100%'})
graph_panel_1 = HBox([graph_1, 
                     ])
graph_panel_2 = HBox([graph_2, 
                     ])

#  Log display
multi_select = SelectMultiple(layout={'width': '100%', 'height': '450px'})
show_logdata = Accordion(children=[multi_select])
show_logdata.set_title(0, 'Show Logdata')

#  Table and popup output
popup_box = Output()
df_output = Output()

#  Compose UI
ui = VBox([
    header,
    graph_panel_1, 
    graph_panel_2, 
    show_logdata
])
display(ui, popup_box)

# --- make widgets responsive ---

for widget in [upload_widget]:
    widget.observe(start, names='value')
    widget.observe(data_reset, names='value')

for widget in [multi_select]:
    widget.observe(draw_graph, names='value')

toggle_popup.observe(update_popup, names='value')

Output()